In [ ]:
#importing libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
#reading dataset

In [4]:
df = pd.read_csv('./cleaned_tweets.csv')


In [5]:
df.head()

,sentiment,text,cleaned_tweet,Porter_Stem,Snowball_Stem
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",aww bummer shoulda got david carr third day,aww bummer shoulda got david carr third day,aww bummer shoulda got david carr third day
1,0,is upset that he can't update his Facebook by ...,upset cannot update facebook texting might cry...,upset can not updat facebook text might cri re...,upset can not updat facebook text might cri re...
2,0,@Kenichan I dived many times for the ball. Man...,dived many times ball managed save rest go bounds,dive mani time ball manag save rest go bound,dive mani time ball manag save rest go bound
3,0,my whole body feels itchy and like its on fire,whole body feels itchy like fire,whole bodi feel itchi like fire,whole bodi feel itchi like fire
4,0,"@nationwideclass no, it's not behaving at all....",behaving im mad cannot see,behav im mad can not see,behav im mad can not see


In [6]:
#drop text

In [7]:
df = df[['sentiment', 'Snowball_Stem']]

In [8]:
df.head()

,sentiment,Snowball_Stem
0,0,aww bummer shoulda got david carr third day
1,0,upset can not updat facebook text might cri re...
2,0,dive mani time ball manag save rest go bound
3,0,whole bodi feel itchi like fire
4,0,behav im mad can not see


In [9]:
#removing rows with nan

In [10]:
df.isna().sum()

sentiment           0
Snowball_Stem    8046
dtype: int64

In [11]:
df = df.dropna()

In [12]:
df.isna().sum()

sentiment        0
Snowball_Stem    0
dtype: int64

In [13]:
#splitting into test and train

In [14]:
y= df.iloc[:,0:1].values
x = df.iloc[:,1].values


In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)

In [16]:
#collectin unique words in corpus

In [17]:
def count_unique_words(tweets):
    unique = Counter()
    for tweet in tweets:
        for word in tweet.split():
            unique[word] += 1
    return unique

In [18]:
word_count = count_unique_words(x_train)

In [19]:
len(word_count)

179447

In [20]:
#reducing dataframe size

In [21]:
reduced_df = pd.concat([df[df.sentiment != 0][:50000], df[df.sentiment == 0][:50000]])
reduced_df.shape
x=reduced_df['Snowball_Stem']
y=reduced_df['sentiment']
x_train,x_test,y_train,y_test = train_test_split(x, y)
x_train.shape, x_test.shape,y_train.shape,y_test.shape

((75000,), (25000,), (75000,), (25000,))

In [22]:
#tfidf unigram

In [23]:
v1 = TfidfVectorizer()
v1.fit(x)
x1_train = v1.transform(x_train)
x1_test = v1.transform(x_test)

In [24]:
#tfdidf bigram

In [25]:
v2 = TfidfVectorizer(ngram_range = (2, 2))
v2.fit(x)
x2_train = v2.transform(x_train)
x2_test = v2.transform(x_test)

In [26]:
#tfidf unigram+bigram

In [27]:
X = df["Snowball_Stem"]
len(X)

1591954

In [28]:
v3 = TfidfVectorizer(ngram_range = (1, 2))
v3.fit(X)
x3_train = v3.transform(x_train)
x3_test = v3.transform(x_test)

In [29]:
#encoding labels

In [30]:
Encoder = LabelEncoder()
y_train = Encoder.fit_transform(y_train)
y_test = Encoder.fit_transform(y_test)

In [31]:
x3_test

<25000x3546142 sparse matrix of type '<class 'numpy.float64'>'
	with 316878 stored elements in Compressed Sparse Row format>

In [32]:
rfc=RandomForestClassifier(n_estimators=10,random_state=0)
rfc.fit(x3_train,y_train)
rfc_pred=rfc.predict(x3_test)


In [33]:
accuracy_score(rfc_pred,y_test)

0.72216

In [34]:
#saving model

In [35]:
import pickle

In [37]:
RFC_model_path = "./RFC_UnigramBigram_72.pickle"
vectorizer_path ="./UnigramBigram_vectorizer2.pickle"
pickle.dump(rfc, open(RFC_model_path, 'wb'))